# AI MEGADADOS 23-1

**NOME**: SEU NOME AQUI!

## Parte 1 - CarTracking!

Na parte 1 da prova de MEGADADOS, iremos trabalhar com a base de dados sintética **CarTracking**.

### Insper autograding!

Para receber feedback dos exercício na parte 1, iremos utilizar o `insper autograding`. Se não fez este passo, abra o notebook disponível na aula 09.

**Sugestão**: para aproveitar o `.env`, crie uma pasta para a prova no mesmo local onde costuma deixar as pastas das **aulas**!

### Instalação da base

Execute os scripts `script_001.sql` e `script_002.sql` no MySQL Workbench. Estes scripts criam uma base **cartracking** e inserem alguns dados de exemplo para resolução da prova.

A base pode ser representada pelo seguinte diagrama do model orelacional (diagrama ER):

<img src="img/deer_cartracing.png">

A base da prova simula um cenário de **rastreamento de veículos**. Nela, estão cadastrados **automóveis** de **clientes**. Os automóveis podem ter múltiplos **rastreadores** instalados, mas cada rastreador está associado a apenas um automóvel.

Os rastreadores irão gerar **eventos** de rastreamento, contendo informações de geolocalização e temperatura em determinados momentos do tempo.

## Como resolver os exercícios?

Crie a base da prova em sua máquina (passo anterior). Utilize o MySQL Workbench ou o conector para testar as queries. Quando estiver bastante certo de que a resposta está correta, faça a submissão para o servidor.

## Import das bibliotecas

Vamos realizar o import das bibliotecas.

In [1]:
import mysql.connector
from functools import partial
import os
import insperautograder.jupyter as ia
from dotenv import load_dotenv

E vamos criar nosso HELPER de conexão com o banco! Perceba que, uma vez configurado o `.env` não precisaremos mais informar usuários, senhas e URLs!

In [2]:
load_dotenv(override=True)

def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host=os.getenv('MD_DB_SERVER'),
        user=os.getenv('MD_DB_USERNAME'),
        password=os.getenv('MD_DB_PASSWORD'),
        database='cartracking',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

### Notas

Para conferir a nota da correção automática da prova, utilize:

In [33]:
ia.grades(task='ai_md_23_1')

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | ai_md_23_1  | ex01        |      1 |     10 |
|  1 | ai_md_23_1  | ex02        |      1 |     10 |
|  2 | ai_md_23_1  | ex03        |      1 |     10 |
|  3 | ai_md_23_1  | ex04        |      1 |     10 |
|  4 | ai_md_23_1  | ex05        |      1 |      0 |
|  5 | ai_md_23_1  | ex06        |      1 |      0 |

'C:\\Users\\carol\\Documents\\Insper\\6_Semestre\\Megadados\\AI\\ai_md_23_1'

In [4]:
os.getenv("IAG_TOKEN")

'iagtok_bc7b7c5f1bed5b0b35ffd58fce82236db5684fdd686cc2eb'

In [5]:
ia.grades(by='TASK', task='ai_md_23_1')

|    | Tarefa     |   Nota |
|---:|:-----------|-------:|
|  0 | ai_md_23_1 |      0 |

**Obs**: Esta parte da avaliação vale **6 pontos** na nota da prova.
- 50% da nota será pela correção automática do servidor
- 50% da nota será pela correção pelo professor, que levará em consideração, por exemplo:
    - Quão distante sua solução ficou de uma solução correta
    - Se as queries estão formatadas:
        - Padrão de maiúsculas e minúsculas
        - Tabulações e quebras de linhas
    - Se utiliza *alias* nas tabelas, especialmente em queries com múltiplas tabelas
    
A nota retornada pelo servidor estará no intervalo `0.0` a `10.0` e será multiplicada por `0.3` para compor os `50%` da correção automática.

Ainda, considere que os testes não são exaustivos. Sua solução deve funcionar para qualquer conjunto de dados no *schema* **cartracking**.

**Exercício 1**: Crie uma query que retorne a quantidade de clientes pessoa jurídica cadastrados na base.

**Obs**: considere que um cliente pessoa jurídica é um cliente com cnpj preenchido!

In [17]:
sql_ex01 = '''
SELECT 
    COUNT(cnpj)
FROM 
    cliente
WHERE
    cnpj IS NOT NULL
'''

db(sql_ex01)

Executando query:
(3,)


Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [18]:
ia.sender(answer='sql_ex01', task='ai_md_23_1', question='ex01', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex01', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 2**: Crie uma query que retorne, **sem repetição**, todos os `modelo` e `marca` dos `automovel` cadastrados. Retorne ordenado de forma crescente pelo `modelo`.

In [32]:
sql_ex02 = '''
SELECT 
     m.modelo, ma.marca
FROM 
    automovel a JOIN modelo m ON a.idmodelo = m.idmodelo
    INNER JOIN marca ma ON m.idmarca = ma.idmarca
    
GROUP BY
    m.modelo
    
ORDER BY
    m.modelo ASC


'''

db(sql_ex02)

Executando query:
('Civic', 'Honda')
('Corolla', 'Toyota')
('Jetta', 'Volkswagen')
('Kicks', 'Nissan')
('RAV4', 'Toyota')
('Sentra', 'Nissan')


Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [31]:
ia.sender(answer='sql_ex02', task='ai_md_23_1', question='ex02', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex02', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 3**: Crie uma query que retorne, o `nome` do `cliente` e a **quantidade** de automóveis que o cliente possui cadastrado na base.


**Obs**:
- Os clientes com mais automóveis cadastrados devem aparecer primeiro.
- Clientes sem automóveis cadastrados não devem ser retornados

In [11]:
sql_ex03 = '''
SELECT
    nome, COUNT(a.idautomovel) as VALOR
FROM
    cliente c INNER JOIN automovel a ON c.idcliente = a.idcliente

GROUP BY
    c.idcliente 
    
ORDER BY 
    VALOR DESC
'''

db(sql_ex03)

Executando query:
('Agromil', 4)
('Universo dos Transportes de Luxo', 3)
('Ana Maria', 1)


Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [12]:
ia.sender(answer='sql_ex03', task='ai_md_23_1', question='ex03', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex03', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 4**: Crie uma query que retorne, nesta ordem, o `idautomovel`, `ano`, `idcliente`, `idmodelo` e `travado` dos automóveis que **não possuem nenhum rastreador** vinculado a eles.


**Obs**: ordene de forma crescente pelo `idautomovel`.

In [13]:
sql_ex04 = '''
SELECT 
    a.idautomovel, a.ano, a.idcliente, a.idmodelo, a.travado
FROM 
    automovel a LEFT OUTER JOIN rastreador r  ON a.idautomovel = r.idautomovel
WHERE 
    r.ativo IS NULL
ORDER BY
    a.idautomovel ASC
'''

db(sql_ex04)

Executando query:
(4, 2018, 5, 7, 1)
(5, 1999, 1, 8, 0)


Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [14]:
ia.sender(answer='sql_ex04', task='ai_md_23_1', question='ex04', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex04', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 5**: Considerando apenas os rastreadores **ativos**, crie uma query que retorne o `nome` do cliente, o `modelo` do automóvel rastreado, a temperatura **máxima** e **média** do motor registradas nos eventos do automóvel.

**Obs**:
- Retorne ordenado pela temperatura máxima, de forma decrescente
- Retorne apenas os registros com temperatura máxima do motor acima de `85`
- A coluna de temperatura máxima deve se chamar `max_temp_motor`
- A coluna de temperatura média deve se chamar `avg_temp_motor`

In [56]:
sql_ex05 = '''
SELECT 
    c.nome, m.modelo,MAX(e.temperatura_motor) AS max_temp_motor, AVG (e.temperatura_motor) AS avg_temp_motor
FROM 
    cliente c INNER JOIN automovel a ON c.idcliente = a.idcliente 
    INNER JOIN rastreador r ON r.idautomovel = a.idautomovel 
    INNER JOIN modelo m ON m.idmodelo = a.idmodelo
    INNER JOIN evento e ON e.idrastreador = r.idrastreador

WHERE 
    r.ativo = 1 AND
    e.temperatura_motor > 85
GROUP BY 
    a.idmodelo,
    c.idcliente
ORDER BY
    max_temp_motor DESC

'''

db(sql_ex05)

Executando query:
('Universo dos Transportes de Luxo', 'Corolla', Decimal('90.20'), Decimal('88.540000'))
('Universo dos Transportes de Luxo', 'Sentra', Decimal('89.92'), Decimal('89.920000'))
('Agromil', 'Jetta', Decimal('89.49'), Decimal('88.530000'))


Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [54]:
ia.sender(answer='sql_ex05', task='ai_md_23_1', question='ex05', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex05', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 6**: Crie uma tabela temporária `tmp_cliente_eventos` contendo o `nome`, `cpf`, `cnpj` do cliente, além da quantidade de **eventos** registrados na tabela `evento` para cada cliente.

Requisitos:
- Retorne ordenado pelo `nome` de forma crescente
- A coluna de contagem deve se chamar `qtde_eventos`
- Clientes sem eventos devem ter o valor `0` em `qtde_eventos`
- Nos clientes com o `cpf` ou `cnpj` nulos, retorne uma string `'NAO_INFORMADO'`

In [20]:
sql_ex06 = '''
SELECT
    c.nome, c.cpf, c.cnpj, COUNT(e.idevento)
FROM 
    cliente c INNER JOIN automovel a ON c.idcliente = a.idcliente
    INNER JOIN rastreador r ON r.idautomovel = a.idautomovel 
    INNER JOIN evento e ON e.idrastreador = r.idrastreador

GROUP BY
    e.idevento

'''

db(sql_ex06)

Executando query:
('Universo dos Transportes de Luxo', None, '31.780.614/0001-67', 1)
('Universo dos Transportes de Luxo', None, '31.780.614/0001-67', 1)
('Universo dos Transportes de Luxo', None, '31.780.614/0001-67', 1)
('Universo dos Transportes de Luxo', None, '31.780.614/0001-67', 1)
('Universo dos Transportes de Luxo', None, '31.780.614/0001-67', 1)
('Agromil', None, '97.861.127/0001-37', 1)
('Agromil', None, '97.861.127/0001-37', 1)
('Agromil', None, '97.861.127/0001-37', 1)
('Universo dos Transportes de Luxo', None, '31.780.614/0001-67', 1)
('Agromil', None, '97.861.127/0001-37', 1)
('Agromil', None, '97.861.127/0001-37', 1)
('Agromil', None, '97.861.127/0001-37', 1)
('Agromil', None, '97.861.127/0001-37', 1)
('Universo dos Transportes de Luxo', None, '31.780.614/0001-67', 1)
('Universo dos Transportes de Luxo', None, '31.780.614/0001-67', 1)
('Universo dos Transportes de Luxo', None, '31.780.614/0001-67', 1)
('Universo dos Transportes de Luxo', None, '31.780.614/0001-67', 1)
(

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [21]:
ia.sender(answer='sql_ex06', task='ai_md_23_1', question='ex06', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex06', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Conferindo as notas!

In [48]:
ia.grades(task='ai_md_23_1')

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | ai_md_23_1  | ex01        |      1 |     10 |
|  1 | ai_md_23_1  | ex02        |      1 |     10 |
|  2 | ai_md_23_1  | ex03        |      1 |     10 |
|  3 | ai_md_23_1  | ex04        |      1 |     10 |
|  4 | ai_md_23_1  | ex05        |      1 |      0 |
|  5 | ai_md_23_1  | ex06        |      1 |      0 |

## Parte 2 - Confraternizar!

Você foi procurado por um grupo de alunos do Insper para uma consultoria na área de dados! Eles reclamaram que precisam realizar diversas confraternizações entre os estudantes do Insper, sendo difícil gerir todos os detalhes relativos aos eventos.

Ainda, é desejo dos organizadores que toda a gestão seja digital, sem mais ingressos no papel!

Em conversas com o cliente, você levantou as seguintes informações sobre o domínio de negócios:
- As confraternizações são agendadas para acontecer em determinada data.
- As confraternizações sempre possuem um tipo (calourada, pizzada, cine-cultural, etc.). Então, uma confraternização sempre é uma nova realização (ou oferecimento) de um tipo de evento.
- Cada confraternização pode ser organizada por diversos alunos. É preciso manter um cadastro dos organizadores, indicando o cargo ocupado em cada festa (quando o aluno fizer parte da organização).
- Os ingressos apenas podem ser adquiridos por alunos. O aluno pode comprar quantos ingresso quiser (para ele e/ou convidados). Quando o aluno compra um ingresso de determinada festa, ele tem a opção de indicar o nome do convidado que irá utilizar o ingresso.

Utilize estas informações para propor soluções adequadas para os próximos exercícios.

**Obs**: Esta parte da avaliação vale **4 pontos** na nota da prova:
- 1.50 para o exercício 7
- 0.75 para o exercício 8
- 0.75 para o exercício 9
- 1.00 para o exercício 10

**Exercício 7**: Desenhe o diagrama do modelo relacional deste problema. Não esqueça de indicar claramente os tipos, chaves primárias, chaves estrangeiras, e a cardinalidade dos relacionamentos.

**Obs:** Salve a imagem do diagrama na pasta `img`. Edite na resposta o caminho para a imagem!

<div class="alert alert-success">
    
Edite na resposta o caminho para a imagem!
    
<img src="img/exemplo.png">

</div>

**Opcional**: caso julgue necessário, utilize o bloco abaixo para justificar decisões tomadas no desenho do diagrama.

<div class="alert alert-success">
    
Seu texto AQUI!

</div>

**Exercício 8**: Cole a **DDL** para criação da tabela de `organizadores`.

<div class="alert alert-success">

```mysql

USE Database;

DROP TABLE IF EXISTS organizadores;

CREATE TABLE organizadores (
id_Organizadores INT PRIMARY KEY NOT NULL,
nome VARCHAR(45) NOT NULL,
cargo VARCHAR(45) NOT NULL,
id_confraternizacoes VARCHAR (30) NOT NULL FOREIGN KEY,
    REFERENCES Confraternizacoes (id_confraternizacoes)
);


```

</div>

**Exercício 9**: Cole a **DML** para inserir um registro na tabela de `organizadores`. Utilize dados quaisquer!

**Obs**: a query precisa estar correta, mas não necessariamente irá rodar no notebook do professor (pois pode depender de chaves estrangeiras). Não é obrigatório (nem vale nota) que você gaste tempo preenchendo valores em outras tabelas para que a query de fato funcione, mas tudo bem caso queira fazer assim!

<div class="alert alert-success">

```mysql

INSERT INTO Database.organizadores (nome, cargo, id_confraternizacoes )
	VALUES ('Carol Chaim', 'Gerente', 1);
```

</div>

**Exercício 10**: Discorra sobre as diferenças entre os relacionamentos identificadores e não-identificadores.

Esta questão será corrigida considerando a seguinte subrica:
| Conceito | Nota | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Apenas citou o assunto ou alguns fatos sem explicações                                                                                                  |
| D        | 0.3 |Explicou superficialmente o assunto ou fatos mas sem muitos detalhes conclusivos                                                                        |
| C        | 0.6 |Explicou com detalhes, apresentando definições concretas.                                                                                               |
| B        | 0.8 |Explicou com detalhes, apresentando definições concretas e exemplos de uso.                                                                            |
| A        | 1.0 |Explicou com detalhes, apresentando definições concretas, exemplos de uso e ainda outros tópicos correlatos, fazendo uma conexão lógica entre eles. |

<div class="alert alert-success">


" Identificador: Relacionamento em que a identidade de uma instancia de uma outra entidade esta atrelada a identidade de outra instancia em outra entidade. Essencialmente quando a chave primaria de uma entidade deveria conter uma chave estrangeira para outra entidade

Nao-identificador: Quando a identidade de uma instancia nao depende intrinsecamente da indentidade de outra instancia." 
    
De modo geral, uma maneira para nos determinarmos qual a relacao de uma tabela para outra é avaliando se uma das tabelas "sobreviveria" sem a existencia da segunda tabela. Um exemplo seria que um pedido nao existeria sem um cliente, e portanto eles sao relacionamento identificadores. Em contrapartida, um produto poderia existir sem  um pedido e portanto seria um relacionamento nao identificador.

</div>

## Entrega!

É hora de entregar. Faça um **zip** do notebook + imagens chamado `ai_megadados.zip` e submeta no Blackboard!

**Obs**:
- `zip`, não use `rar`!
- Não precisa entregar os scripts fornecidos pelo professor (pasta scripts)